# Import and parsing of the calrep 2-port file format.
Written by AWS 04/01/2016


## The 2-port format is closely related to the one-port format.
1. It contains 3 tables, each one is a one-port table for S11, S21, and S22 respectively. 
2. The data is in Magnitude-Angle format with several columns for error analysis
3. The tables have names inside of the .asc file like TABLE 1 : Freq GHz,  S11
4. There are also A,B,C tables that are the constituents of the .asc file, they are directly one-port format




In [26]:
import os
import re
import fnmatch
# All of the data types dealing with Calrep are in this module
import pyMeasure.Code.DataHandlers.NISTModels as NISTModels
import pyMeasure.Code.DataHandlers.TouchstoneModels as TouchstoneModels
from pyMeasure.Code.DataHandlers.GeneralModels import *


The module smithplot was not found,please put it on the python path


In [2]:
# We set the directory to the one with .asc and the tables in it 
home_directory=r'C:\share\ascii.dut'
# We willl start by getting all the .txt files
os.chdir(home_directory)
file_names=os.listdir(home_directory)
file_names=fnmatch.filter(file_names,'*.txt')
print file_names

['000146a.txt', '000146b.txt', '000146c.txt', '000889a.txt', '000889b.txt', '000889c.txt', '001136a.txt', '001136b.txt', '001136c.txt', '001371a.txt', '001371b.txt', '001371c.txt', '001452a.txt', '001452b.txt', '001452c.txt', '02806.txt', '052101.txt', '060127.txt', '079538a.txt', '079538b.txt', '079538c.txt', '079552a.txt', '079552b.txt', '079552c.txt', '08046A.txt', '08047A.txt', '700035a.txt', '700035b.txt', '700083a.txt', '700083b.txt', '700084a.txt', '700084b.txt', '700621a.txt', '700621b.txt', '811687a.txt', '811687b.txt', '811825a.txt', '811825b.txt', 'M105P1.txt', 'M105P2.txt', 'M110P2.txt', 'MY6070a.txt', 'MY6070b.txt', 'MY6070c.txt', 'MY6733a.txt', 'MY6733b.txt', 'MY6733c.txt', 'MY6777a.txt', 'MY6777b.txt', 'MY6777c.txt', 'N101P1.txt', 'N110P1.txt', 'N202FWa.txt', 'N202FWb.txt', 'N202FWc.txt', 'N202RVa.txt', 'N202RVb.txt', 'N202RVc.txt', 'N203FWa.txt', 'N203FWb.txt', 'N203FWc.txt', 'N205FWa.txt', 'N205FWb.txt', 'N205FWc.txt', 'N205RVa.txt', 'N205RVb.txt', 'N205RVc.txt', 'NTN1

In [3]:
# Now the 2 port files are all the ones with an a, b, and c table. 
two_port_files=[]
for file_name in file_names:
    two_port_match=re.search('(?P<two_port_name>\w+)c.txt',file_name,re.IGNORECASE)
    if two_port_match:
        root_name=two_port_match.groupdict()['two_port_name']
        two_port_files.append(root_name)
print two_port_files

['000146', '000889', '001136', '001371', '001452', '079538', '079552', 'MY6070', 'MY6733', 'MY6777', 'N202FW', 'N202RV', 'N203FW', 'N205FW', 'N205RV', 'NTN204', 'NTN205', 'NTN206']


## Each root name has 4 files asscoiated with it
1. a .asc file with all of the tables in it
2. a .txt file with a appended to it, that is S11 in Comma Separated Values
3. a .txt file with b appended to it, that is S21 in CSV
4. a .txt file with c appended to it, that is S22 in CSV

In [4]:
# to import the file we use the OnePortModel
S11_table=NISTModels.OnePortModel('000146a.txt')
S21_table=NISTModels.OnePortModel('000146b.txt')
# The C tables have an extra comma at the end of the line so to deal with that we add an option when we open it
options={"row_end_token":',\n'}
S22_table=NISTModels.OnePortModel('000146c.txt',**options)
# and remove it 
S22_table.options["row_end_token"]=None
#print S21_table

In [ ]:
#print S22_table

In [5]:
def ascii_data_table_join(column_selector,table_1,table_2):
    """Given a column selector (name or zero based index) and 
    two tables a data_table with extra columns is returned. The options from table 1 are inherited
    headers and footers are added, if the tables have a diffferent number of rows problems may occur"""
    if table_1.header is None and table_2.header is None:
        header=None
    elif table_1.header is None:
        header=table_2.header[:]
    elif table_2.header is None:
        header=table_2.header[:]
    else:
        header=[]
        for line in table_1.header:
            header.append(line)
        for line in table_2.header:
            header.append(line)
            
    if table_1.footer is None and table_2.footer is None:
        footer=None
    elif table_1.footer is None:
        footer=table_2.footer[:]
    elif table_2.header is None:
        footer=table_2.footer[:]
    else:
        footer=[]
        for line in table_1.footer:
            footer.append(line)
        for line in table_2.footer:
            footer.append(line)
        
    if column_selector in table_2.column_names:
        column_selector_2=table_2.column_names.index(column_selector)
        
    options=table_1.options.copy()
    new_table=AsciiDataTable(None,**options)
    new_table.data=table_1.data[:]
    new_table.column_names=table_1.column_names[:]
    if header is None:
        new_table.header=None
    else:
        new_table.header=header[:]
    if footer is None:
        new_table.footer=None
    else:
        new_table.footer=footer[:]
    #Todo: make this work for tables without column_names
    for index,column in enumerate(table_2.column_names):
        if column == table_2.column_names[column_selector_2]:
            pass
        else:
            if table_2.options["column_types"] is None:
                column_type=None
            else:
                if type(table_2.options["column_types"]) is DictionaryType:
                    column_type=table_2.options["column_types"][column]
                elif type(table_2.options["column_types"]) is ListType:
                    column_type=table_2.options["column_types"][index]
            column_data=table_2.get_column(column)  
            new_table.add_column(column,column_type=column_type,column_data=column_data)

    return new_table
    

In [6]:
for index,column in enumerate(S11_table.column_names):
    if column is not 'Frequency':
        S11_table.column_names[index]='S11_'+column
#print S11_table

In [7]:
for index,column in enumerate(S21_table.column_names):
    if column is not 'Frequency':
        S21_table.column_names[index]='S21_'+column
#print S21_table

In [8]:
for index,column in enumerate(S22_table.column_names):
    if column is not 'Frequency':
        S22_table.column_names[index]='S22_'+column
#print S22_table

In [9]:
table=ascii_data_table_join('Frequency',S11_table,S21_table)
#print table

In [10]:
print table.options["row_formatter_string"]

{0:.4f}{delimiter}{1:.4f}{delimiter}{2:.4f}{delimiter}{3:.4f}{delimiter}{4:.4f}{delimiter}{5:.4f}{delimiter}{6:.2f}{delimiter}{7:.2f}{delimiter}{8:.2f}{delimiter}{9:.2f}{delimiter}{10:.2f}{delimiter}{11}{delimiter}{12}{delimiter}{13}{delimiter}{14}{delimiter}{15}{delimiter}{16}{delimiter}{17}{delimiter}{18}{delimiter}{19}{delimiter}{20}


In [16]:
#print table

In [14]:
joined_table_all=ascii_data_table_join('Frequency',table,S22_table)

In [17]:
#print joined_table_all

In [18]:
# Now the question is if we can do this for .asc files also
asc_file='000146.asc'
in_file=open(asc_file,'r')
lines=[]
tables=["Table 1","Table 2","Table 3"]
begin_lines=[]
for index,line in enumerate(in_file):
    lines.append(line)
    for table in tables:
        if re.search(table,line,re.IGNORECASE):
            begin_lines.append(index)
in_file.close()

In [19]:
begin_lines

[4, 347, 690]

In [24]:
header_lines=lines[0:3]
S11_lines=lines[7:346]
S21_lines=lines[350:689]
S22_lines=lines[693:]

In [21]:
header_lines

['000146\n', '29 Jan 2016\n', '\n']

In [25]:
S21_lines

[' 0.1000  0.0019  0.0035  0.0005  0.0000  0.0072     21.67  72.00    0.03    2.02  144.01\n',
 ' 0.1500  0.0024  0.0035  0.0005  0.0001  0.0070     17.13  60.29    0.03    1.56  120.60\n',
 ' 0.2000  0.0026  0.0034  0.0005  0.0001  0.0069      8.78  57.30    0.03    0.53  114.60\n',
 ' 0.2500  0.0030  0.0034  0.0005  0.0001  0.0069      3.49  51.65    0.03    0.75  103.29\n',
 ' 0.3000  0.0031  0.0034  0.0005  0.0001  0.0069     -3.51  50.43    0.03    0.85  100.86\n',
 ' 0.3500  0.0033  0.0034  0.0005  0.0001  0.0069    -11.51  47.75    0.03    0.78   95.49\n',
 ' 0.4000  0.0034  0.0034  0.0005  0.0001  0.0069    -18.81  46.78    0.03    0.54   93.55\n',
 ' 0.4500  0.0036  0.0034  0.0005  0.0001  0.0068    -26.59  45.48    0.03    0.83   90.96\n',
 ' 0.5000  0.0035  0.0034  0.0005  0.0001  0.0068    -31.45  45.88    0.03    0.65   91.76\n',
 ' 0.5500  0.0036  0.0034  0.0005  0.0001  0.0068    -40.05  45.54    0.03    0.71   91.09\n',
 ' 0.6000  0.0036  0.0034  0.0005  0.0001  0.0068 

In [29]:
row_pattern=TouchstoneModels.make_row_match_string(["Frequency", "Magnitude", "uMb", "uMa", "uMd", "uMg", "Phase","uPhb", "uPha", "uPhd", "uPhg"])

In [32]:
match=re.match(row_pattern,S11_lines[0])
match.groupdict()

{'Frequency': '0.1000',
 'Magnitude': '0.0023',
 'Phase': '29.36',
 'uMa': '0.0005',
 'uMb': '0.0035',
 'uMd': '0.0001',
 'uMg': '0.0072',
 'uPha': '0.03',
 'uPhb': '64.96',
 'uPhd': '1.38',
 'uPhg': '129.93'}

In [69]:
column_names=["Frequency", "Magnitude", "uMb", "uMa", "uMd", "uMg", "Phase","uPhb", "uPha", "uPhd", "uPhg"]
row_pattern=TouchstoneModels.make_row_match_string(column_names)
def parse_lines(string_list,**options):
    """Default behavior returns a two dimensional list given a list of strings that represent a table."""
    defaults={"row_pattern":None,"column_names":None,
              "column_types":None,"output":'list_list',"delimiter":None,"row_begin_token":None,
              "row_end_token":None,"data_begin_token":None,"data_end_token":None,"escape_character":None}
    parse_options={}
    for key,value in defaults.iteritems():
        parse_options[key]=value
    for key,value in options.iteritems():
        parse_options[key]=value
    out_list=[]
    out_dict_list=[]
    out_list=strip_tokens(string_list,*[parse_options["data_begin_token"],parse_options["data_begin_token"]])
    out_list=strip_all_line_tokens(out_list,begin_token=parse_options["row_begin_token"],
                                  end_token=parse_options["row_end_token"])
    #print("{0} is {1}".format('out_list',out_list))
    try:
        if parse_options["row_pattern"]:
            parsed_out_list=[]
            for line in out_list:
                match=re.match(parse_options["row_pattern"],line)
                if parse_options["column_names"] and match:
                    out_row_dict=match.groupdict()
                    out_dict_list.append(out_row_dict)
                    out_row=[]
                    for column_name in parse_options["column_names"]:
                        out_row.append(out_row_dict[column_name])
                    #print("{0} is {1}".format('out_row',out_row))
                    parsed_out_list.append(out_row)
            out_list=parsed_out_list
        else:
            out_list=split_all_rows(out_list,delimiter=parse_options["delimiter"],
                                     escape_character=parse_options["escape_character"])
        if parse_options["column_types"]:
            out_list=convert_all_rows(out_list,parse_options["column_types"])
        if out_dict_list == [] and parse_options["column_names"] is not None:
            for row in out_list:
                out_row_dict={column_name:row[index] for index,column_name in parse_options["column_names"]}
    except:
        print("Could not parse table")
        raise
    if parse_options["output"] in ['list_list']:
        return out_list
    elif parse_options["output"] in ['dict_list']:
        return out_dict_list
    elif parse_options["output"] in ['numpy']:
        # Todo: Add the conversion to numpy array
        return out_list
    elif parse_options["output"] in ['pandas']:
        # Todo: Add the conversion to pandas
        return out_list

In [74]:
column_types=['float' for i in range(len(column_names))]
options={"row_pattern":row_pattern,"column_names":column_names,"output":"list_list"}
options["column_types"]=column_types
S11_data=parse_lines(S11_lines,**options)
S21_data=parse_lines(S21_lines,**options)
S22_data=parse_lines(S22_lines,**options)

strip_line_tokens failed to strip None,None from 



In [65]:
type(data[0]["Frequency"])

str

In [76]:
#S22_data

## Now we can create 3 Ascii data tables and then as